# 1. After creating the stream link in Qgis, resample it to the DEM (reference )

In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling as RioResampling
from rasterio.transform import from_origin
import numpy as np
import os

# Input files
streamlink_path = r"E:\DEMs\Beamon_Creek_Boyer_River\Streamlink.tif"
template_rst_path = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\Beamon_creek_fill.rst"
output_rst_path = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\StreamLink_aligned.rst"

# Open the template .rst (Beamon_creek_fill) to get georeferencing
with rasterio.open(template_rst_path) as template:
    template_meta = template.meta.copy()
    template_crs = template.crs
    template_transform = template.transform
    template_shape = (template.height, template.width)

# Open the streamlink raster
with rasterio.open(streamlink_path) as src:
    src_data = src.read(1)
    src_transform = src.transform
    src_crs = src.crs

    # Prepare destination array with correct shape and dtype
    destination = np.zeros(template_shape, dtype=np.int16)

    # Reproject to align with template raster
    reproject(
        source=src_data,
        destination=destination,
        src_transform=src_transform,
        src_crs=src_crs,
        dst_transform=template_transform,
        dst_crs=template_crs,
        resampling=Resampling.nearest
    )

# Update metadata for output
template_meta.update({
    'driver': 'RST',
    'dtype': 'int16',
    'count': 1,
    'nodata': 0  # use 0 if background is streamless
})

# Save as aligned .rst
with rasterio.open(output_rst_path, 'w', **template_meta) as dst:
    dst.write(destination, 1)

print("✅ Aligned StreamLink raster saved to:", output_rst_path)


# 2. I like to check all rasters for their properties 

In [ ]:
import rasterio
import os

# List of raster paths
raster_paths = [
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\StreamLink_aligned.rst",
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\Beamon_creek_fill.rst",
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\Beamon_Rfactor.rst",
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\CFactor.rst",
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\KFactor.rst",
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\KTCmap.rst",
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\parcel_map.rst",
    r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\PFactor.rst"
]

# Print summary
for path in raster_paths:
    with rasterio.open(path) as src:
        print(f"📄 {os.path.basename(path)}")
        print(f"  - CRS:        {src.crs}")
        print(f"  - Size:       {src.width} x {src.height} (cols x rows)")
        print(f"  - Pixel size: {src.transform.a} x {-src.transform.e} (X x Y)")
        print(f"  - Dtype:      {src.dtypes[0]}")
        print(f"  - NoData:     {src.nodata}")
        print("-" * 50)


# 3. Adjacent txt creation

In [ ]:
import rasterio
import numpy as np
import pandas as pd
from collections import defaultdict

# Load stream segment raster
with rasterio.open(r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\StreamLink_aligned.rst") as src:
    segments = src.read(1)
    transform = src.transform

rows, cols = segments.shape
neighbors = [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (-1,1), (1,-1), (1,1)]
connections = set()

# Loop over raster pixels
for row in range(rows):
    for col in range(cols):
        seg_id = segments[row, col]
        if seg_id <= 0:
            continue
        for dr, dc in neighbors:
            r, c = row + dr, col + dc
            if 0 <= r < rows and 0 <= c < cols:
                neighbor_id = segments[r, c]
                if neighbor_id > 0 and neighbor_id != seg_id:
                    connections.add((seg_id, neighbor_id))

# Remove upstream duplicates (only retain downstream flow)
# (this is optional and requires flow direction logic — can be added later)

# Save to tab-separated file
df = pd.DataFrame(list(connections), columns=["from", "to"])
df.to_csv("adjacent_segments.txt", sep="\t", index=False)
# ...existing code...
output_path = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\adjacent_segments.txt"
df.to_csv(output_path, sep="\t", index=False)
print(f"Saved adjacent segments to: {output_path}")

# 4. Upstream txt creation

In [ ]:
import pandas as pd

# Path to previous adjacent segment file (from → to)
adjacent_path = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\adjacent_segments.txt"
upstream_path = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\upstream_segments.txt"

# Load adjacent segments
df_adj = pd.read_csv(adjacent_path, sep="\t")

# Reverse direction: to = edge, from = upstream_edge
df_up = df_adj.rename(columns={"from": "upstream_edge", "to": "edge"})

# Add proportion = 1.0
df_up["proportion"] = 1.0

# Reorder columns
df_up = df_up[["edge", "upstream_edge", "proportion"]]

# Save to tab-delimited file
df_up.to_csv(upstream_path, sep="\t", index=False)

print(f"✅ Upstream segment table written to:\n{upstream_path}")


# 5. River routing raster creation

In [ ]:
import rasterio
import numpy as np

# Load base stream raster to get shape and transform
with rasterio.open(r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\StreamLink_aligned.rst") as src:
    meta = src.meta.copy()
    shape = src.shape

# Create blank routing raster
routing = np.zeros(shape, dtype=np.int16)

# Example: impose routing value 6 at a certain location
routing[50, 100] = 6  # row 50, col 100 = flow to lower-left

# Save to disk at your desired location
output_path = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Input\river_routing_map.rst"
meta.update(dtype='int16', nodata=0)
with rasterio.open(output_path, "w", **meta) as dst:
    dst.write(routing, 1)

print(f"Saved routing raster to: {output_path}")